In [6]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib import style
style.use('ggplot')
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [36]:
train = pd.read_csv('/content/Coursera Review.csv')
train.head()

,Review,Label
0,good and interesting,0
1,"This class is very helpful to me. Currently, I...",0
2,like!Prof and TAs are helpful and the discussi...,0
3,Easy to follow and includes a lot basic and im...,0
4,Really nice teacher!I could got the point eazl...,0


In [37]:
def remove_pattern(text,pattern):

    # re.findall() finds the pattern i.e @user and puts it in a list for further task
    r = re.findall(pattern,text)

    # re.sub() removes @user from the sentences in the dataset
    for i in r:
        text = re.sub(i,"",text)

    return text

In [39]:
train['Tidy_Reviews'] = np.vectorize(remove_pattern)(train['Review'], "@[\w]*")
train['Tidy_Reviews'] = train['Tidy_Reviews'].str.replace("[^a-zA-Z#]", " ")
train['Tidy_Reviews'] = train['Tidy_Reviews'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
train.head()

<ipython-input-39-5f4cfd360a71>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train['Tidy_Reviews'] = train['Tidy_Reviews'].str.replace("[^a-zA-Z#]", " ")


,Review,Label,Tidy_Reviews
0,good and interesting,0,good interesting
1,"This class is very helpful to me. Currently, I...",0,This class very helpful Currently still learni...
2,like!Prof and TAs are helpful and the discussi...,0,like Prof helpful discussion among students qu...
3,Easy to follow and includes a lot basic and im...,0,Easy follow includes basic important technique...
4,Really nice teacher!I could got the point eazl...,0,Really nice teacher could point eazliy


In [40]:
tokenized_review = train['Tidy_Reviews'].apply(lambda x: x.split())
tokenized_review.head()

0                                  [good, interesting]
1    [This, class, very, helpful, Currently, still,...
2    [like, Prof, helpful, discussion, among, stude...
3    [Easy, follow, includes, basic, important, tec...
4        [Really, nice, teacher, could, point, eazliy]
Name: Tidy_Reviews, dtype: object

In [41]:
from nltk import PorterStemmer
ps = PorterStemmer()
tokenized_review = tokenized_review.apply(lambda x: [ps.stem(i) for i in x])
tokenized_review.head()

0                                     [good, interest]
1    [thi, class, veri, help, current, still, learn...
2    [like, prof, help, discuss, among, student, qu...
3    [easi, follow, includ, basic, import, techniqu...
4        [realli, nice, teacher, could, point, eazliy]
Name: Tidy_Reviews, dtype: object

In [42]:
for i in range(len(tokenized_review)):
    tokenized_review[i] = ' '.join(tokenized_review[i])

train['Tidy_Reviews'] = tokenized_review
train.head()

,Review,Label,Tidy_Reviews
0,good and interesting,0,good interest
1,"This class is very helpful to me. Currently, I...",0,thi class veri help current still learn thi cl...
2,like!Prof and TAs are helpful and the discussi...,0,like prof help discuss among student quit acti...
3,Easy to follow and includes a lot basic and im...,0,easi follow includ basic import techniqu sketchup
4,Really nice teacher!I could got the point eazl...,0,realli nice teacher could point eazliy


In [43]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')

# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(train['Tidy_Reviews'])

df_bow = pd.DataFrame(bow.todense())

df_bow

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107014,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_df=0.90, min_df=2,max_features=1000,stop_words='english')
tfidf=tfidf.fit_transform(train['Tidy_Reviews'])
df_tfidf = pd.DataFrame(tfidf.todense())
df_tfidf

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
train_bow = bow[:107018]

train_bow.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [51]:
train_tfidf= tfidf[:107018]

train_tfidf.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [52]:
from sklearn.model_selection import train_test_split

In [53]:
x_train_bow,x_valid_bow,y_train_bow,y_valid_bow = train_test_split(train_bow,train['Label'],test_size=0.3,random_state=2)
x_train_tfidf,x_valid_tfidf,y_train_tfidf,y_valid_tfidf = train_test_split(train_tfidf,train['Label'],test_size=0.3,random_state=17)

In [54]:
pip install keras.preprocessing

In [55]:
from keras.preprocessing.text import one_hot, Tokenizer
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from numpy import array
import tensorflow as tf

In [56]:
x_train_bow = x_train_bow.toarray()
x_valid_bow = x_valid_bow.toarray()

In [57]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(units=16, activation='relu', input_dim=x_train_bow.shape[1]))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train_bow, y_train_bow, batch_size=5, epochs=15)

Epoch 1/15
14983/14983 [==============================] - 63s 4ms/step - loss: 0.1914 - accuracy: 0.9556
Epoch 2/15
14983/14983 [==============================] - 36s 2ms/step - loss: 0.1774 - accuracy: 0.9558
Epoch 3/15
14983/14983 [==============================] - 42s 3ms/step - loss: 0.1770 - accuracy: 0.9562
Epoch 4/15
14983/14983 [==============================] - 43s 3ms/step - loss: 0.1716 - accuracy: 0.9571
Epoch 5/15
14983/14983 [==============================] - 39s 3ms/step - loss: 0.1642 - accuracy: 0.9587
Epoch 6/15
14983/14983 [==============================] - 40s 3ms/step - loss: 0.1538 - accuracy: 0.9614
Epoch 7/15
14983/14983 [==============================] - 41s 3ms/step - loss: 0.1482 - accuracy: 0.9629
Epoch 8/15
14983/14983 [==============================] - 37s 2ms/step - loss: 0.1480 - accuracy: 0.9636
Epoch 9/15
14983/14983 [==============================] - 40s 3ms/step - loss: 0.1448 - accuracy: 0.9641
Epoch 10/15
14983/14983 [==============================

In [58]:
score = model.evaluate(x_valid_bow, y_valid_bow, verbose=1)

1004/1004 [==============================] - 3s 2ms/step - loss: 0.1580 - accuracy: 0.9627


In [59]:
x_train_tfidf = x_train_tfidf.toarray()
x_valid_tfidf = x_valid_tfidf.toarray()

In [60]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(units=16, activation='relu', input_dim=x_train_bow.shape[1]))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train_tfidf, y_train_tfidf, batch_size=5, epochs=15)

Epoch 1/15
14983/14983 [==============================] - 62s 4ms/step - loss: 0.1620 - accuracy: 0.9558
Epoch 2/15
14983/14983 [==============================] - 40s 3ms/step - loss: 0.1512 - accuracy: 0.9558
Epoch 3/15
14983/14983 [==============================] - 36s 2ms/step - loss: 0.1508 - accuracy: 0.9558
Epoch 4/15
14983/14983 [==============================] - 36s 2ms/step - loss: 0.1514 - accuracy: 0.9558
Epoch 5/15
14983/14983 [==============================] - 40s 3ms/step - loss: 0.1516 - accuracy: 0.9560
Epoch 6/15
14983/14983 [==============================] - 37s 2ms/step - loss: 0.1494 - accuracy: 0.9562
Epoch 7/15
14983/14983 [==============================] - 34s 2ms/step - loss: 0.1506 - accuracy: 0.9565
Epoch 8/15
14983/14983 [==============================] - 35s 2ms/step - loss: 0.1504 - accuracy: 0.9564
Epoch 9/15
14983/14983 [==============================] - 40s 3ms/step - loss: 0.1489 - accuracy: 0.9562
Epoch 10/15
14983/14983 [==============================

In [61]:
score = model.evaluate(x_valid_tfidf, y_valid_tfidf, verbose=1)

1004/1004 [==============================] - 2s 2ms/step - loss: 0.1745 - accuracy: 0.9551
